In [ ]:
import pandas as pd
import os
import re
import requests
print(os.getcwd())

In [ ]:
path_00_scraping = '../../../00_Scraping/'
path_01_database = '../../../01_Database_Construction/'

----------------------------------------------
### Journal Summary Statistics

The following code blocks generate code (and output tables) that help to answer the following questions:
- How many articles are there?
  - Across our 15 journals (we have to build ALJ into the workflow) there are 23,097 articles
- How many articles are there per journals?
  - See the tables generated by papers_database_summaries.R
  - This can be found as modified .tex file in scopus_econlit_matching_results_out.tex OR embedded in the report


In [ ]:
articles_path = path_01_database + 'papers/fuzzy_matched_papers_df.csv'
articles_df = pd.read_csv(articles_path)
print(articles_df.columns)
print(len(articles_df))
articles_df.head(5)


----
### Authors Summary Statistics
- How many unique authors do we observe?
  - We have 52,461 author-article observations and 18,167 unique authors.
- How many articles are there per author
- Who are the top authors in each journal?


In [ ]:
authors_path = path_01_database + 'authors/authors_df.csv'
authors_df = pd.read_csv(authors_path)
print(authors_df.columns)
print(len(authors_df))
authors_df.head(5)

In [ ]:
unique_authors_count = len(authors_df.drop_duplicates(subset=['sc_author_id']))
print('Unique author count: {}'.format(unique_authors_count))

articles_per_author_df = authors_df[['sc_issn', 'sc_pub_name', 'sc_author_id', 'sc_author_given_name', 'sc_author_last_name']]

author_names_df = articles_per_author_df.drop_duplicates(subset='sc_author_id')[['sc_author_id', 'sc_author_given_name', 'sc_author_last_name']]
pub_names_df = articles_per_author_df.drop_duplicates(subset='sc_issn')[['sc_issn', 'sc_pub_name']]
# print(author_names_df)



articles_per_author = articles_per_author_df.groupby(by=['sc_author_id'])['sc_issn'].count()
articles_per_author = articles_per_author.to_frame().reset_index()
articles_per_author = articles_per_author.rename(columns={
    'sc_issn' : 'a_per_author_total'
})
print('Number of unique authors: {}'.format(len(articles_per_author)))

journal_articles_per_author =  articles_per_author_df.groupby(by=['sc_author_id', 'sc_issn'])['sc_pub_name'].count()
journal_articles_per_author = journal_articles_per_author.to_frame().reset_index()
journal_articles_per_author = journal_articles_per_author.rename(columns={
    'sc_pub_name' : 'a_per_author_per_journal'
})

print('Number of unique author-journals: {}'.format(len(journal_articles_per_author)))


per_author_df = pd.merge(journal_articles_per_author, articles_per_author, how='left', on='sc_author_id')
per_author_df = pd.merge(per_author_df, author_names_df, how='left', on='sc_author_id')
per_author_df = pd.merge(per_author_df, pub_names_df, how='left', on='sc_issn')

per_author_df.to_csv('tables/input/articles_per_author.csv', index=False, encoding='utf-8')

per_author_df
